In this notebook, we will build a 3D map of a scene from a small set of images and refine it with the featuremetric optimization. We then localize an image downloaded from the Internet and show the effect of the refinement.

# Setup
We start by defining some output paths: where the intermediate files will be stored.

In [ ]:
%load_ext autoreload
%autoreload 2
import tqdm, tqdm.notebook
tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import os
import time
import sys
from hloc import extract_features, match_features, reconstruction, pairs_from_exhaustive, visualization
from hloc.visualization import plot_images, read_image
from hloc.utils.viz_3d import init_figure, plot_points, plot_reconstruction, plot_camera_colmap

from pixsfm.util.visualize import init_image, plot_points2D
from pixsfm.refine_hloc import PixSfM
from pixsfm import ostream_redirect
from PIL import Image, ImageDraw
import pycolmap
#import visualize_model
# redirect the C++ outputs to notebook cells
cpp_out = ostream_redirect(stderr=True, stdout=True)
cpp_out.__enter__()

In [ ]:
import torch 
print(torch.__version__)
print(torch.cuda.get_arch_list())

In [ ]:
images = Path('datasets/monarch/')
outputs = Path('outputs/monarch-demo/')
!rm -rf $outputs
sfm_pairs = outputs / 'pairs-sfm.txt'
loc_pairs = outputs / 'pairs-loc.txt'
features = outputs / 'features.h5'
matches = outputs / 'matches.h5'
raw_dir = outputs / "raw"
ref_dir = outputs / "ref"
''' model location in case of intrinsics locked '''
ref_dir_locked = outputs / "ref_locked"
''' model location in case of intrinsics not locked '''
ref_dir_not_locked = outputs / "ref_dir_not_locked" 

Here we will use SuperPoint local features with the SuperGlue matcher, but it's easy to switch to other features like SIFT or R2D2.

In [ ]:
feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

# 3D mapping and refinement
First we list the images used for mapping. These are all day-time shots of Sacre Coeur.

In [ ]:
# '''masking of the tractor hood from the images '''
# ''' output => datasets/monarch/{target_folder}/image_name.jpg '''
# def draw_box_around_tractor_hood(image_path, target_folder): 
#     image = Image.open(image_path)
#     w, h = image.size
#     box_x1, box_y1 = 460, 770  # Top-left corner
#     box_x2, box_y2 = 1630, 1080  # Bottom-right corner
#     outline_color = (0, 0, 0)  # Red in RGB format
#     fill_color = (0, 0, 0)  # Black in RGB format
#     draw = ImageDraw.Draw(image)
#     draw.rectangle([box_x1, box_y1, box_x2, box_y2], outline=outline_color, fill=fill_color)
#     directory_path,filename = os.path.split(image_path)
#     parent_directory_path = os.path.dirname(directory_path)
#     target_directory = os.path.join(parent_directory_path, target_folder)
#     os.makedirs(target_directory, exist_ok = True)
#     target_image_path = os.path.join(target_directory,filename)
#     image.save(target_image_path)
#     return target_image_path 

In [ ]:
references_left = [str(p.relative_to(images)) for i, p in enumerate((images / 'left/').iterdir())]
references_right = [str(p.relative_to(images)) for i, p in enumerate((images / 'right/').iterdir())]

In [ ]:
print(len(references_left))

In [ ]:
references_left = sorted(references_left, key=lambda x: int(x.split('/')[-1].split('.')[0]))
references_right = sorted(references_right, key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [ ]:
print(len(references_left))

In [ ]:
references_left = references_left[40:82] 
references_right = references_right[40:82]
references = references_left + references_right

In [ ]:
'''sorting references so that each stereo pair is together in the list '''
references = sorted(references, key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [ ]:
print(references)

In [ ]:
# ''' masking the tractor hood in all the images'''
# ''' returns list of path to the masked images '''
# start_time = time.time()
# target_folder = "masked_images"
# masked_references = [draw_box_around_tractor_hood(p, target_folder) for p in references]
# end_time = time.time()
# elapsed_time = end_time - start_time

# target_path = os.path.join(images, target_folder)

# ''' sorting masked_references sequentially '''
# ''' smf => sorted masked references '''
# #smf = sorted(masked_references, key = lambda x: int(((x.split("/")[-1]).split(".")[0]).split("_")[0]))

# print(f"type(masked_references): {type(masked_references)}")


In [ ]:
# smf = []
# for i in range(0, len(references)//2 - 1): 
#     left  = "masked_images/" + str(i) + "_left.jpg"
#     right = "masked_images/" + str(i) + "_right.jpg"
#     smf.append(left)
#     smf.append(right)

In [ ]:
# print(f"smf: {smf}")
# print(f"len(smf) : {len(smf)}")

In [ ]:
print(references)

In [ ]:
references[59]

In [ ]:
features_path_ = extract_features.main(feature_conf, images, image_list= references, feature_path=features)
#match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

In [ ]:
from hloc.extract_features import list_h5_names
h5_feature_names = list_h5_names(features_path_)
print(f"len(h5_feature_names): {len(h5_feature_names)}")
print(h5_feature_names[:10])

Then we extract features and match them across image pairs. Since we deal with few images, we simply match all pairs exhaustively.

In [ ]:
# features_path_ = extract_features.main(feature_conf, images, image_list=references_final, feature_path=features)
# #match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

In [ ]:
from hloc.utils.viz import plot_keypoints, save_plot
from hloc.utils.io import get_keypoints
import matplotlib.pyplot as plt
import numpy as np
import os

ref_trim_ = references[:4]
plot_images([read_image(images / r) for r in ref_trim_], dpi=50, figsize=4.2)

kps_list_ = [] 
for r in ref_trim_:
    kps = get_keypoints(features_path_, r)
    print(type(kps))
    kps_list_.append(kps)
    
plot_keypoints(kps_list_, colors = "red",  ps = 10)

current_path_ = os.getcwd()

print("current_path: ", current_path_)

print(type(current_path_))

final_path = current_path_ + "/kps.png"


save_plot(final_path)


In [ ]:
import collections.abc as collections
isinstance(references, collections.Iterable)

In [ ]:
pairs_from_exhaustive.stereo_main(sfm_pairs, image_list=references)

In [ ]:
#print("features: ", features)
#print("matches: ", matches)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

In [ ]:
print(len(list_h5_names(matches)))

In [ ]:
print(list_h5_names(features))

In [ ]:
match_names = list_h5_names(matches)
for name in match_names: 
    if "right-52.jpg" in name: 
        print (name)

In [ ]:
''' script to plot matches between two frames'''
from hloc.utils.viz import plot_matches
from hloc.utils.io import get_matches, get_keypoints
#img1 = images.joinpath(references[0])
#img2 = images.joinpath(references[1])

#print(f"img1 : {img1.as_posix()} img_2: {img2.as_posix()}")

print(f"features: {features}")
kp1 = get_keypoints(features, references[0])
kp2 = get_keypoints(features, references[1])
print(f"kp1.shape: {kp1.shape}")

m, _ = get_matches(matches, references[0], references[1])
print(f"m.shape: {m.shape}")

m1 = np.array([kp1[i] for i in m[:,0]])
m2 = np.array([kp2[i] for i in m[:, 1]])

#print(m1[:10])

plot_images([read_image(images / r) for r in references[:2]], dpi=50, figsize=4.2)
#plot_matches(kp1.transpose, kp2.transpose)
#plot_matches(kp1.transpose, kp2.transpose)
plot_matches(m1, m2)
#plot_matches(m[:,0], m[:,1])
#print(m[:10])
#kp1 = 
#matches, scores = 

Now we run the reconstruction with and without the featuremetric refinement. For this dataset, when computing the dense features, we resize the images such that they are not larger than 1024 pixels.

In [ ]:
fx = 1093.2768
fy = 1093.2768
cx = 964.989
cy = 569.276
opencv_camera_params =','.join(map(str, (fx, fy, cx, cy, 0, 0, 0, 0)))

In [ ]:
#sfm = PixSfM({"dense_features": {"max_edge": 1024}})


#conf1 = {"dense_features": {"max_edge": 1024}}

conf2 = {
    "BA": {"optimizer": {"refine_focal_length": False,"refine_extra_params": False, "refine_extrinsics": False}},
    "dense_features": {"max_edge":1024}
}

sfm = PixSfM(conf=conf2)



In [ ]:
'''CASE 1 => INITIAL K IS PROVIDED + K IS NOT LOCKED '''
'''
image_options = dict(camera_model='OPENCV', 
                    camera_params=opencv_camera_params,
                    )
mapper_options_locked = dict(ba_refine_focal_length=False, 
                      ba_refine_extra_params=False,
                     ba_refine_principal_point=False)

hloc_args_locked = dict(image_list=references,
                image_options=image_options,
                # mapper_options=mapper_options_locked,
                camera_mode="PER_FOLDER")

K_locked, sfm_outputs_locked = sfm.reconstruction(ref_dir_not_locked, images, sfm_pairs, features, matches, **hloc_args_locked)
'''

In [ ]:
'''CASE 2 => INITIAL K IS PROVIDED + K IS LOCKED '''

image_options = dict(camera_model='OPENCV', 
                     camera_params=opencv_camera_params
                    )

mapper_options_one = dict(ba_refine_focal_length=False, 
                      ba_refine_extra_params=False,
                     ba_refine_principal_point=False)

mapper_options_two = dict(ba_refine_focal_length=False, 
                      ba_refine_extra_params=False,
                     ba_refine_principal_point=False)

hloc_args_not_locked = dict(image_list=references,
                image_options=image_options,
                camera_mode="PER_FOLDER",
                mapper_options=mapper_options_two)

#hloc_args_not_locked = dict(image_list=references)

K_locked, sfm_outputs_not_locked = sfm.reconstruction(ref_dir_locked, images, sfm_pairs, features, matches, **hloc_args_not_locked)


In [ ]:
'''
e_lw => left camera pose in world frame (4 * 4)
e_rw => right camera pose in world frame (4 * 4)
'''
#def calculate_relative_pose(e_lw, e_rw):
def calculate_relative_pose(e_lw: np.ndarray, e_rw: np.ndarray):
    #print(f"Inside the calculate_relative_pose function")
    from scipy.spatial.transform import Rotation
    e_wl = np.linalg.inv(e_lw)
    #print(f"e_wl: {e_wl}")
    #e_rl = e_rw * np.linalg.inv(e_lw) #right camera in the frame of the left camera
    #e_rl = e_rw * e_wl #right camera in the frame of the left camera
    #print(f"e_rl: {e_rl}")
    e_rl = np.dot(e_rw,np.linalg.inv(e_lw))
    R = e_rl[:3,:3] #extracting the rotation matrix
    dx = e_rl[0,3]
    dy = e_rl[1,3]
    dz = e_rl[2,3]
    dquat = Rotation.from_matrix(R).as_quat()
    #rel_pose =  [dx, dy] + dquat
    rel_pose = [dx,dy,dz]
    for q in dquat: 
        rel_pose.append(q)
    return rel_pose
    #return [dx,dy]
    #print(f"dx: {dx} dy: {dy} dquat: {dquat}")


def cam_extrinsics(img):
    from read_write_model import qvec2rotmat
    R = qvec2rotmat(img.qvec)
    t = img.tvec.reshape(3,-1)
    #print(f"R: {R} t: {t}")
    R_t = np.concatenate((R,t), axis = 1)
    #R_t = np.vstack([np.array([0,0,0,1]), R_t])
    R_t = np.vstack([R_t, np.array([0,0,0,1])])
    return R_t    #  4 * 4 matrix

def calculate_relative_pose_between(left_idx: int, right_idx: int):
    left_img = sparse_img_dict[left_idx]
    right_img = sparse_img_dict[right_idx]
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    rel_pose = calculate_relative_pose(e_lw, e_rw)
    return rel_pose


    

In [ ]:
from read_write_model import qvec2rotmat
qvec = [1,0,0,0]
R = qvec2rotmat(qvec)
print(f"R: {R}")

#### Camera positions WITHOUT Rig Bundle Adjustment

In [ ]:
from pathlib import Path
#sparse_dir = Path("/home/skumar/stereo_colmap_cli_output/sparse/")
#sparse_dir = ref_dir_locked / "hloc"
sparse_dir = Path("/home/skumar/stereo_colmap_cli_output/")
print(f"sparse_dir: {sparse_dir.as_posix()}")
sparse_images = sparse_dir / "images.bin"
sparse_points3D = sparse_dir / "points3D.bin"
sparse_cameras = sparse_dir / "cameras.bin"

In [ ]:
import sys
sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import read_images_binary 
sparse_img_dict = read_images_binary(sparse_images)
print(f"{len(sparse_img_dict.keys())} ==> {sparse_img_dict.keys()}")
print(f"min_key: {min(sparse_img_dict.keys())} mx_key: {max(sparse_img_dict.keys())}")

In [ ]:
cam_extrinsics(sparse_img_dict[1])

In [ ]:
import numpy as np
rel_poses = []
num_images = len(sparse_img_dict.keys())
for idx in range(1, num_images // 2 + 1):
    left_img = sparse_img_dict[idx]
    right_img = sparse_img_dict[idx + 42]
    #print(f"left_img_name: {left_img.name} right_img_name: {right_img.name}")
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    e_rl = calculate_relative_pose(e_lw, e_rw)
    rel_poses.append(e_rl)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.notebook_repr_html', True)
df = pd.DataFrame(rel_poses, columns=['dx', 'dy', 'dz', 'qx' , 'qy', 'qz' , 'qw'])
df.style

In [ ]:
#dr = np.hstack((np.array(df['dx']).reshape(-1,1), np.array(df['dy']).reshape(-1,1), np.array(df['dz']).reshape(-1,1)))

In [ ]:
#dr.shape

In [ ]:
# x = np.linalg.norm(dr, axis=1, ord=2)
# plt.hist(x, 100)
# plt.show()

#### Camera poses with Rig Bundle Adjustment

In [ ]:
rig_ba_sparse_dir = Path("/home/skumar/rig_dense/sparse/")
print(f"rig_ba_sparse_dir: {rig_ba_sparse_dir.as_posix()}")
rig_ba_sparse_images = rig_ba_sparse_dir / "images.bin"
rig_ba_sparse_points3D = rig_ba_sparse_dir / "points3D.bin"
rig_ba_sparse_cameras = rig_ba_sparse_dir / "cameras.bin"

In [ ]:
import sys
sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import read_images_binary 
rig_ba_sparse_img_dict = read_images_binary(rig_ba_sparse_images)
print(f"{len(rig_ba_sparse_img_dict.keys())} => {rig_ba_sparse_img_dict.keys()}")

In [ ]:
import numpy as np
rig_ba_rel_poses = []
num_images = len(rig_ba_sparse_img_dict.keys())
for idx in range(1, num_images // 2 + 1):
    left_img = rig_ba_sparse_img_dict[idx]
    right_img = rig_ba_sparse_img_dict[idx + 42]
    if idx < 5:
        print(f"left_img_name: {left_img.name} right_img_name: {right_img.name}")
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    rel_pose = calculate_relative_pose(e_lw, e_rw)
    rig_ba_rel_poses.append(rel_pose)

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.notebook_repr_html', True)
df = pd.DataFrame(rig_ba_rel_poses, columns=['dx', 'dy', 'dz', 'qx' , 'qy', 'qz' , 'qw'])
df.style

We now plot the reconstructions side-by-side. We can click on the legend entries to toggle them.

In [ ]:
#curr_directory = os.getcwd()
#dense_model = Path("outputs/mvs/")
from pathlib import Path
import os
import pycolmap
#ply_path = Path("/home/skumar/colmap_output/fused.ply")
#bin_path_1 = Path("/home/skumar/pixel-perfect-sfm/outputs/monarch-demo/ref_dir_not_locked/hloc/")
#bin_path_2 = Path("/home/skumar/pixel-perfect-sfm/outputs/monarch-demo/ref_dir_not_locked/")

#bin_path = Path("/home/skumar/colmap_cli_output/")
locked_path = Path("/home/skumar/colmap_cli_output/")
#not_locked_path = ref_dir_locked
#ply_model = pycolmap.Reconstruction()
#ply_model.import_PLY(ply_path.as_posix())

#bin_model1 = pycolmap.Reconstruction()
#bin_model1.read_binary(bin_path_1.as_posix())

#bin_model2 = pycolmap.Reconstruction()
#bin_model2.read_binary(bin_path_2.as_posix())

#bin_model = pycolmap.Reconstruction()
#bin_model.read_binary(bin_path.as_posix())

locked_model = pycolmap.Reconstruction()
locked_model.read_binary(locked_path.as_posix())

#not_locked_model = pycolmap.Reconstruction()
#not_locked_model.read_binary(not_locked_path.as_posix())


In [ ]:
#print(f"ply_model.summary(): {ply_model.summary()}")
#print(f"bin_model.summary(): {bin_model.summary()}")

print(f"locked_model.summary(): {locked_model.summary()}")
#print(f"not_locked_model.summary(): {not_locked_model.summary()}")


In [ ]:

fig3d = init_figure()
args = dict(max_reproj_error=3.0, min_track_length=2, cs=1.2)
#plot_reconstruction(fig3d, locked_model, color='rgba(255, 0, 0, 0.5)', name="K_not_locked", **args)
#plot_reconstruction(fig3d, K_locked, color='rgba(0, 255, 0, 0.5)', name="K_locked", **args)
plot_reconstruction(fig3d, locked_model, color='rgba(255, 255, 0, 0.5)', name="K_", **args)
#plot_reconstruction(fig3d, locked_model, color='rgba(255,0,0,0.5)', name="locked_model")

fig3d.show()


In [ ]:
sys.path.insert(1, "/home/skumar/colmap/scripts/python/")
points  = read_write_model.read_points3D_binary(Path("/home/skumar/colmap_cli_output/points3D.bin"))

In [ ]:
print(f"type(points): {type(points)}")
#print(f"list(point.keys(): {list(points.keys()}))
key_list = list(points.keys())
print(f"key_list[0:10] : {key_list[0:10]}")

In [ ]:
print(points[155652])

In [ ]:
sys.path.insert(1, "/home/skumar/colmap/scripts/python/")
import read_write_fused_vis
vis_model = read_write_fused_vis.read_fused("/home/skumar/colmap_cli_output/fused.ply", "/home/skumar/colmap_cli_output/fused.ply.vis")

In [ ]:
print(f"type(vis_model) : {type(vis_model)}")
print(f"len(vis_model) : {len(vis_model)}")
vis_model[0]

In [ ]:
#print(f"bin_model1.summary(): {bin_model1.summary()}")
#print(f"bin_model1.summary(): {bin_model1.summary()}")


In [ ]:
import sys
sys.path.append("/home/skumar/colmap/scripts/python")
import read_write_fused_vis
f1 = Path("/home/skumar/colmap_cli_output/fused.ply")
f2 = Path("/home/skumar/colmap_cli_output/fused.ply.vis")
#os.path.exists(f1)
#os.path.exists(f2)
#df = read_write_fused_vis.read_fused(f1,f2)
df = read_write_fused_vis.read_fused(f1.as_posix(),f2.as_posix())
#@r@ead_write_fused_vis.read_fused(f1.as_posix(), f2.as_posix())
print(df[0])

In [ ]:
from read_write_model import read_images_binary
p1 = Path("/home/skumar/colmap_cli_output/images.bin")
os.path.exists(p1)
p2 = Path("/home/skumar/colmap_cli_output/sparse/images.bin")
os.path.exists(p2)
d1 = read_images_binary(p1.as_posix())
d2 = read_images_binary(p2.as_posix())

In [ ]:
print(f"type(d1): {type(d1)}")
print(f"d1.keys(): {d1.keys()}")

In [ ]:
print(type(df))
print(df[0])

In [ ]:
#print(K_not_locked.summary())
print(dense_reconstruction.summary())

In [ ]:
fig3d = init_figure()
args = dict(max_reproj_error=30.0, min_track_length=2, cs=1.2)
#plot_reconstruction(fig3d, K_locked, color='rgba(255, 0, 0, 0.5)', name="K_locked", **args)
plot_reconstruction(fig3d, bin_model, color='rgba(0, 255, 0, 0.5)', name="K_not_locked", **args)

fig3d.show()

In [ ]:
dense_reconstruction.export_PLY("dense_reconstruction.ply")

In [ ]:
max_reproj_error = 3.0
min_track_length=2
rec = dense_reconstruction
p3Ds = [p3D for _, p3D in rec.points3D.items()]
    
xyzs = [p3D.xyz for p3D in p3Ds]
print(xyzs)

In [ ]:
fig3d = init_figure()
args = dict(max_reproj_error=3.0, min_track_length=2, cs=1)
#plot_reconstruction(fig3d, K_locked_dense, color='rgba(0, 255, 0, 0.5)', name="K_locked", **args)
plot_reconstruction(fig3d, x, color='rgba(255, 255, 0, 0.5)', name="K_not_locked", **args)

fig3d.show()

In [ ]:
fig3d = init_figure()
args = dict(max_reproj_error=30.0, min_track_length=2, cs=1)
plot_reconstruction(fig3d, K_locked_dense, color='rgba(0, 255, 0, 0.5)', name="K_locked", **args)
#plot_reconstruction(fig3d, K_not_locked, color='rgba(255, 255, 0, 0.5)', name="K_not_locked", **args)

fig3d.show()

We can also visualize the detected keypoints (blue) and the final reprojections (red) for a given mapping image. You can drag to zoom in. As you can see, the points were refined by a few pixels at most but the 3D points and camera poses can be refined up to a few meters.

In [ ]:
img = refined.images[refined.reg_image_ids()[0]]
cam = refined.cameras[img.camera_id]
fig = init_image(images / img.name)    
plot_points2D(fig, [p2D.xy for p2D in img.points2D if p2D.has_point3D()])
plot_points2D(fig, cam.world_to_image(img.project(refined)), color='rgba(255, 0, 0, 0.5)')
fig.show()

## PointCloud Segmentation
Segementation of the dense pointcloud

In [ ]:
import sys
sys.path.append("/home/skumar/colmap/scripts/python")

### Analysing Sparse Pointcloud Data

In [ ]:
sparse_dir = Path("/home/skumar/bkp/stereo_colmap_cli_output/sparse/")
print(f"sparse_dir: {sparse_dir.as_posix()}")
sparse_images = sparse_dir / "images.bin"
sparse_points3D = sparse_dir / "points3D.bin"
sparse_cameras = sparse_dir / "cameras.bin"

In [ ]:
sparse_model = pycolmap.Reconstruction()
sparse_model.read_binary(sparse_dir.as_posix())
print(f"sparse_model.summary(): {sparse_model.summary()}")

In [ ]:
sparse_model.export_PLY("sparse_model.ply")

### Loading Dense PointCloud BIN Files

In [ ]:
dense_dir = Path("/home/skumar/rig_stereo_colmap_cli_output/")
dense_images = dense_dir / "images.bin"
dense_points3D = dense_dir / "points3D.bin"
dense_cameras = dense_dir / "cameras.bin"

In [ ]:
dense_colmap = pycolmap.Reconstruction()
dense_colmap.read_binary(dense_dir.as_posix())
print(f"summary: {dense_colmap.summary()}")

In [ ]:
dense_colmap.export_PLY("rig_sparse.ply")

In [ ]:
fig3d = init_figure()
args = dict(max_reproj_error=3.0, min_track_length=2, cs=20)
#plot_reconstruction(fig3d, dense_colmap, color='rgba(255, 255, 0, 0.5)', name="dense_colmap", **args)
plot_reconstruction(fig3d, dense_colmap, color='rgba(255, 255, 0, 0.5)', name="dense_colmap", **args)
fig3d.show()

### Analysing Dense PointCloud BIN Files

In [ ]:
'''Analysing images.bin'''
from read_write_model import read_images_binary
a = read_images_binary(dense_images)
#a = read_write_model.read_images_binary(dense_images)
print(type(a))
#print(f"min_key: {min(a.keys())} mx_key: {max(a.keys())}")
#print(a[1])
#print(a[1])


In [ ]:
'''Analysing sparse/images.bin'''
from read_write_model import read_images_binary
a = read_images_binary(sparse_images)
#a = read_write_model.read_images_binary(dense_images)
print(type(a))
#print(f"min_key: {min(a.keys())} mx_key: {max(a.keys())}")
#print(a[1])
#print(a[1])
print(a.keys())
print(type(a[29]))
print(a[29])
print(f"dir(a[29]): {dir(a[29])}")

In [ ]:
print(f"dir(dense_colmap): {dir(dense_colmap)}")

In [ ]:
print(f"dir(dense_colmap.images: {dir(dense_colmap.images)}")
#print(f"dir(dense_colmap.images: {dir(dense_colmap.images.items)}")
#print(f"dir(dense_colmap.images: {dir(dense_colmap.images.keys)}")
#print(f"dir(dense_colmap.images: {dir(dense_colmap.images.values)}")

print(f"type(dense_colmap.images: {type(dense_colmap.images)}")
#print(f"type(dense_colmap.images.items: {type(dense_colmap.images.items)}")
#print(f"type(dense_colmap.images.keys: {type(dense_colmap.images.keys)}")
#print(f"type(dense_colmap.images.values: {type(dense_colmap.images.values)}")



In [ ]:
for key, value in dense_colmap.images.items():
    print(f"Key: {key}, Value: {value}")

In [ ]:
key_list = dense_colmap.images.keys()
print(f"len(key_list): {len(key_list)} type(key_list): {type(key_list)}")
#print(key_list[:10])
for k in key_list: 
    print(k)

In [ ]:
for key, value in a.items():
    print(f"key: {key} a[key].id: {a[key].id} a[key].name: {a[key].name}")


In [ ]:
'''Analysing cameras.bin'''
from read_write_model import read_cameras_binary
b = read_cameras_binary(dense_cameras)
print(f"b.keys(): {list(b.keys())}")
for k, v in b.items():
    v_dict  = v._asdict()
    for kk, vv in v_dict.items():
        print(f"{kk}:{vv}")
        
    

In [ ]:
from read_write_model import read_points3D_binary
points3D_bin = read_points3D_binary(dense_points3D)
print(type(c))
print(f"mn: {min(c.keys())} mx: {max(c.keys())}")
print(f"type(c[6904] : {type(c[6904])}")
for k, v in c[6904]._asdict().items():
    print(f"{k}: {v}")

    
    
    


In [ ]:
dense_model = pycolmap.Reconstruction()
dense_model.read_binary(dense_dir.as_posix())

print(f"dense_model.summary(): {dense_model.summary()}")

In [ ]:
from pprint import pprint
print((dense_model.num_reg_images))

### Loading Dense PointCloud PLY Files

In [ ]:
sys.path.append("/home/skumar/colmap/scripts/python/")
from read_write_fused_vis import read_fused

dense_ply = dense_dir / "fused.ply"
dense_ply_vis = dense_dir / "fused.ply.vis"
dense_ply_model = read_fused(dense_ply.as_posix(), dense_ply_vis.as_posix()) 


In [ ]:
print(f"type(dense_ply_model): {type(dense_ply_model)}")

In [ ]:
print(dense_ply_model[0])

In [ ]:
mx_idx = -1
for pt3d in tqdm(dense_ply_model):
    idx_list = pt3d.visible_image_idxs
    #print(f"type(idx_list): {type(idx_list)}")
    mx_idx_in_list = max(idx_list)
    mx_idx = max(mx_idx, mx_idx_in_list)
    #print(f"mx_idx_in_list: {mx_idx_in_list}")
print(f"mx_idx: {mx_idx}")

In [ ]:
import numpy as np
c_1r = np.array([[1,0,0,0], 
                [0,1,0,1], 
                [0,0,1,2],
                [0, 0, 0, 1]])
c_2r = np.array([[1,0,0,3], 
                [0,1,0,4], 
                [0,0,1,5],
                [0,0,0,1]])

In [ ]:
c_1w = np.array([[1,0,0,0], 
                [0,1,0,0], 
                [0,0,1,0],
                [0,0,0,1]])
c_2w = np.array([[1,0,0,3], 
                [0,1,0,3], 
                [0,0,1,3],
                [0,0,0,1]])


In [ ]:
c_rw = np.dot(np.linalg.inv(c_1r), c_1w)
print(c_rw)

In [ ]:
c_rw = np.dot(np.linalg.inv(c_2r), c_2w)
print(c_rw)

### Analysing Dense PointCloud PLY files

In [ ]:
print(dense_ply_model[0])

### 3D to 2D mapping Helpers

In [ ]:
import numpy as np 
fx = 1093.2768
fy = 1093.2768
cx = 964.989
cy = 569.276
A = np.array([[fx,0 , cx], [0, fy, cy], [0 , 0, 1]]).astype(np.float64)

def get_camera_matrix(fx, fy, cx, cy):
    return np.array([[fx,0 , cx], [0, fy, cy], [0 , 0, 1]]).astype(np.float64)

'''
A => Camera Matrix => 3 * 3
R => Rotation Matrix => 3 * 3
t => Translation Vector => 3 * 1
P => Projection Matrix => 3 * 4
'''
def getP(A, R, t):
    assert A.shape == (3,3)
    assert R.shape == (3, 3)
    assert t.shape == (3,1)
    R_t = np.concatenate((R,t), axis = 1)
    
    #print(f"A.shape: {A.shape}")
    #print(f"R_t.shape: {R_t.shape}")
    #print(f"t.shape: {t.shape}")
    P = np.dot(A ,R_t)
    return P
'''
coords => 3 * 1 => (x, y, 1)
'''
def mask_value_at(img, coords):
    mask_dir = images / "segmentations"
    segmentation_masks = []
    name = img.split('/')[-1] + ".png"
    mask_name = mask_dir.joinpath(name)
    mask_img = cv2.imread(mask_name.as_posix(), cv2.IMREAD_GRAYSCALE)
    coords = list(coords[:, 0])
    return mask_img[int(coords[0]), int(coords[1])]
    pass

In [ ]:
first5pairs = {k: sparse_img_dict[k] for k in list(sparse_img_dict)[:2]}
print(f"type(sparse_img_dict): {type(sparse_img_dict)}")
print(f"type(first5pairs): {type(first5pairs)}")


In [ ]:
print(list(sparse_img_dict.items())[:2])

In [ ]:
list(sparse_img_dict.keys())[29]

In [ ]:
print(sparse_keys[:2])

In [ ]:
sys.path.append("/home/skumar/colmap/scripts/python")
from read_write_model import qvec2rotmat
from tqdm import tqdm
import cv2
import numpy as np
from progress.bar import Bar
from read_write_model import read_images_binary
#print(f"sample_image: {images_bin[1]}")

no_mask_list = []
mask_dict = {}
mask_mapping_dict = {}

#images_bin = read_images_binary(dense_images)
dense_img_dict = read_images_binary(dense_images)
sparse_img_dict = read_images_binary(sparse_images)

#sparse_img_dict_list = list(sparse_img_dict)

sparse_keys = list(sparse_img_dict.keys())

print(f"sparse_keys: {sparse_keys}")

for point3d in tqdm(dense_ply_model): 
    #print(f"point3d.position: {type(point3d.position)} position[0]: {type(point3d.position[0])}")
    mask_found = False
    mask_list = []
    X = np.append(point3d.position, 1).reshape(4,-1)
    #print(f"X: {X} X.shape: {X.shape} type(X): {type(X)}")
    image_ids = point3d.visible_image_idxs
    #print(f"image_ids: {image_ids}")
    for image_id in image_ids: 
        #curr_img = sparse_img_dict[sparse_keys[image_id]]
        curr_img = sparse_img_dict[image_id + 1]
        R = qvec2rotmat(curr_img.qvec)
        t = curr_img.tvec.reshape(3,-1)
        P = getP(A, R, t)
        x = np.dot(P , X) #project X onto image
        x = x / x[2,0]
        if x[0,0] < 1080 and x[0,0] > 0 and x[1,0] < 1920 and x[1,0] > 0:
            curr_img_mask = mask_value_at(curr_img.name, x)
            mask_found = True
            key = (point3d.position[0], point3d.position[1], point3d.position[2])
            #print(f"key: {key} type(key): {type(key)}")
            mask_dict[key] = curr_img_mask
            if curr_img_mask not in mask_mapping_dict: 
                mask_mapping_dict[curr_img_mask] = []
            mask_mapping_dict[curr_img_mask].append(key)
            #print(f"curr_img_mask: {curr_img_mask}")
            break;
        else:
            pass
    if not mask_found:
        #print(f"Could not find a mask for point : {point3d.position}")
        no_mask_list.append(point3d.position)
    
        

In [ ]:
print(f"len(no_mask_list): {len(no_mask_list)}")

In [ ]:
print(f"len(no_mask_list): {len(no_mask_list)}  {len(no_mask_list)/ len(dense_ply_model) * 100}%")

In [ ]:
color_mask_dict = {0: 'rgb(0,0,0)',
                   1: 'rgb(246,4,228)',
                   2: 'rgb(173, 94, 48)',
                   3: 'rgb(68, 171, 117)',
                   4: 'rgb(162, 122, 174)',
                   5: 'rgb(121, 119, 148)',
                   6: 'rgb(253, 75, 40)',
                   7: 'rgb(170,60,100)',
                   9: 'rgb(170,100,60)'    
}


In [ ]:
mask_dict_list = list(mask_dict.keys())
#print(mask_dict_list)
pts3D_list = []
for pts in mask_dict_list:
    #print(f"pts: {pts} type(pts): {type(pts)} type(pts[0]): {type(pts[0])}")
    pts3D_list.append(np.asarray(pts))
#print(pts3D_list)
pts3D_list = np.asarray(pts3D_list)
#print(pts3D_list)

In [ ]:
invalid_points = no_mask_list
invalid_points = np.asarray(invalid_points)
fig3d = init_figure()
plot_points(fig3d, invalid_points)
fig3d.show()

In [ ]:
fig3d = init_figure()

vine_stem_pts = None

for key in mask_mapping_dict.keys():
    '''if(len(mask_mapping_dict[key]) == 0):
        print("0 value")
        continue
    '''
    if key != 4:
        continue
    print(f"key: {key} color: {color_mask_dict[key]} len(mask_mapping_dict[key] : {len(mask_mapping_dict[key])}")
    pts_list = mask_mapping_dict[key]
    pts = []
    for pt in pts_list: 
        pts.append(np.asarray(pt))
    pts = np.asarray(pts)
    print(f"type(pts) : {type(pts)} pts.shape: {pts.shape}")
    vine_stem_pts = pts 
    #print(f"key: {key} pts.shape: {pts.shape} type(pts): {type(pts)} type(pts[0,0]): {type(pts[0,0])}")
    #plot_points(fig3d, pts, color_mask_dict[key], name=str(key))
args = dict(max_reproj_error=3.0, min_track_length=2, cs=1.2)
plot_reconstruction(fig3d, dense_colmap, color='rgba(255, 255, 0, 0.5)', name="dense_colmap", **args)
#plot_points(fig3d, invalid_points, color='rgba(255,0,0,0.5)', name="invalid_points")

fig3d.show()

In [ ]:
print(f"type(vine_stem_pts): {type(vine_stem_pts)} vine_stem_pts.shape: {vine_stem_pts.shape}")

In [ ]:
#xyz = np.random.rand(100, 3)

def write_as_ply(arr): 
    import open3d as o3d
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(arr)
    #o3d.io.write_point_cloud("./data.ply", pcd)
    o3d.visualization.draw_geometries([pcd])


In [ ]:
def get_distance(p1:np.array, p2:np.array):
    return np.linalg.norm(p1-p2)

In [ ]:
p1 = np.array([-0.79,15.4116])
p2 = np.array([-3.27,15.89])

In [ ]:
dis = get_distance(p1,p2)
print(f"dis: {dis}")

### Plotting Segemented Pointcloud

In [ ]:
fig3d = init_figure()
plot_points(fig3d, pts3D_list, name ="k1")
plot_points(fig3d, pts3D_list, name ="k2")
fig3d.show()

In [ ]:
test_map = pycolmap.Reconstruction()
test_map.read_binary("/home/skumar/colmap_cli_output")
plot_reconstruction(test_map?)

### Baseline check

In [ ]:
from read_write_model import read_images_binary
image_dict = read_images_binary(dense_images)
print(image_dict.keys())

In [ ]:
def cam_extrinsics(img):
    from read_write_model import qvec2rotmat
    R = qvec2rotmat(img.qvec)
    t = img.tvec.reshape(3,-1)
    R_t = np.concatenate((R,t), axis = 1)
    R_t = np.vstack([np.array([0,0,0,1]), R_t])
    return R_t    #  4 * 4 matrix
    

In [ ]:
'''
e_lw => left camera pose in world frame (4 * 4)
e_rw => right camera pose in world frame (4 * 4)
'''
def calculate_relative_pose(e_lw, e_rw): 
    from scipy.spatial.transform import Rotation
    e_rl = e_rw * np.linalg.inv(e_lw) #right camera in the frame of the left camera
    R = e_rl[:3,:3] #extracting the rotation matrix
    dx = e_rl[0,3]
    dy = e_rl[1,3]
    dz = e_rl[2,3]
    dquat = Rotation.from_matrix(R).as_quat()
    #rel_pose =  [dx, dy] + dquat
    rel_pose = [dx,dy,dz]
    for q in dquat: 
        rel_pose.append(q)
    return rel_pose
    #return [dx,dy]
    #print(f"dx: {dx} dy: {dy} dquat: {dquat}")

In [ ]:
'''pose of cam1 in the frame of rig'''
e_1r = np.array([[1, 0, 0, 0.06], 
                [0, 1, 0, 0],
                [0, 0, 1, 0], 
                [0, 0, 0, 1]])

''' pose of rig in the frame of cam1'''
e_r1 = np.linalg.inv(e_1r)

print(e_r1)

In [ ]:
np.dot(e_r1 ,e_1r)

In [ ]:
rel_poses = []
for idx in range(0, num_images - 1,2): 
    '''
    if idx > 1:
        break
    '''
    left_img = sparse_img_dict[idx + 1]
    right_img = sparse_img_dict[idx + 2]
    e_lw = cam_extrinsics(left_img)  #left camera pose w.r.t. world
    e_rw = cam_extrinsics(right_img) #right camera pose w.r.t world
    rel_pose = calculate_relative_pose(e_lw, e_rw)
    rel_poses.append(rel_pose)

In [ ]:
print (rel_poses[0])

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.notebook_repr_html', True)
df = pd.DataFrame(rel_poses, columns=['dx', 'dy', 'dz', 'qx' , 'qy', 'qz' , 'qw'])
df.style

In [ ]:
import pandas as pd
import numpy as np

np.random.seed(24)
df = pd.DataFrame({'A': np.linspace(1, 10, 10)})
df = pd.concat([df, pd.DataFrame(np.random.randn(10, 4), columns=list('BCDE'))],
               axis=1)
df.iloc[0, 2] = np.nan
df.style

# Localization
Now that we have a 3D map of the scene, we can localize any image. To demonstrate this, we download [a night-time image from Wikimedia](https://commons.wikimedia.org/wiki/File:Paris_-_Basilique_du_Sacr%C3%A9_Coeur,_Montmartre_-_panoramio.jpg).

In [ ]:
url = "https://upload.wikimedia.org/wikipedia/commons/5/53/Paris_-_Basilique_du_Sacr%C3%A9_Coeur%2C_Montmartre_-_panoramio.jpg"
# try other queries by uncommenting their url
# url = "https://upload.wikimedia.org/wikipedia/commons/8/8e/Sacr%C3%A9_C%C5%93ur_at_night%21_%285865355326%29.jpg"
# url = "https://upload.wikimedia.org/wikipedia/commons/c/c0/La_basilique_du_Sacr%C3%A9-Coeur_au_cr%C3%A9puscule_%28Paris%29_%284147593805%29.jpg"
query = 'query/night.jpg'
!mkdir -p $images/query && wget $url -O $images/$query -q
plot_images([read_image(images / query)], dpi=75)

Again, we extract features for the query and match them exhaustively with all mapping images that were successfully reconstructed.

In [ ]:
references_registered = [refined.images[i].name for i in refined.reg_image_ids()]
extract_features.main(feature_conf, images, image_list=[query], feature_path=features, overwrite=True)
pairs_from_exhaustive.main(loc_pairs, image_list=[query], ref_list=references_registered)
match_features.main(matcher_conf, loc_pairs, features=features, matches=matches, overwrite=True);

We read the EXIF data of the query to infer a rough initial estimate of camera parameters like the focal length. Then we estimate the absolute camera pose using PnP+RANSAC and refine the camera parameters. Under the hood, the `QueryLocalizer` takes care of extracting dense features for the query and runs the keypoint and pose adjustments, QKA and QBA. The refinement refines the camera parameters in-place so we can inspect them.

In [ ]:
import pycolmap
from pixsfm.localize import QueryLocalizer, pose_from_cluster

camera = pycolmap.infer_camera_from_image(images / query)
ref_ids = [refined.find_image_with_name(r).image_id for r in references_registered]
conf = {
    "dense_features": sfm.conf.dense_features,  # same features as the SfM refinement
    "PnP": {  # initial pose estimation with PnP+RANSAC
        'estimation': {'ransac': {'max_error': 12.0}},
        'refinement': {'refine_focal_length': True, 'refine_extra_params': True},
    },
    "QBA": {  # query pose refinement
        "optimizer:": {'refine_focal_length': True, 'refine_extra_params': True},
    }
}
dense_features = sfm_outputs["feature_manager"]
localizer = QueryLocalizer(refined, conf, dense_features=dense_features)
ret, log = pose_from_cluster(localizer, query, camera, ref_ids, features, matches, image_path=images/query)

print(f'found {sum(ret["inliers"])}/{len(ret["inliers"])} inlier correspondences.')
visualization.visualize_loc_from_log(images, query, log, refined, top_k_db=1)

We visualize the correspondences between the query images a few mapping images. We can also visualize the estimated camera pose in the 3D map, shown here in blue.

In [ ]:
pose = pycolmap.Image(tvec=ret['tvec'], qvec=ret['qvec'])
plot_camera_colmap(fig3d, pose, camera, color='rgba(128,128,255,0.5)', name=query, legendgroup="refined")
fig3d.show()